# importations

In [1]:
import pyxdf
import numpy as np
import matplotlib.pyplot as plt
import joblib
from stft import *

In [2]:
features = joblib.load('features/time_frequency_features.sav')
print(features.keys())
print(features['stft'].stft ['Zxx'].shape, features['ricker'].shape, len(features['y']))

dict_keys(['stft', 'ricker', 'y'])
(3945, 8, 129, 11) (3945, 8, 15, 1250) 3945


# splitting dataset

In [3]:
'''
    use scikit stratified shuffle split method to create train and test dataset
    from eeg['morlet_wavelet'] as X and eeg['y'] as y or labels
'''
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split

ricker_train, ricker_test, y_train, y_test = train_test_split(features['ricker'], np.array(features['y']), test_size=0.3, random_state=42)
print(ricker_train.shape, y_train.shape, ricker_test.shape, y_test.shape)

(2761, 8, 15, 1250) (2761,) (1184, 8, 15, 1250) (1184,)


In [6]:

del X_train,X_test,y_train,y_test

In [ ]:
## after doing all in the above cells, stop here and let me review your CNN knowledge and vhoice

In [12]:
import keras
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.callbacks import History 
history = History()
 
img_x = 249
img_y = 249
img_z = 8
input_shape = (img_x, img_y, img_z)
 
# num_classes = 6
batch_size = 8
num_classes = 7
epochs = 3
 
x_train = x_train.astype('float32')
X_train = x_train[:-n]
X_val = x_train[-n:]

x_test = x_test.astype('float32')
 
y_train = keras.utils.to_categorical(y_train, num_classes)
Y_train = y_train[:-n]
Y_val = y_train[-n:]

y_test = keras.utils.to_categorical(y_test, num_classes)

In [13]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
 

In [15]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
 
 
model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=2,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[history])
 
train_score = model.evaluate(x_train, y_train, verbose=0)
print('Train loss: {}, Train accuracy: {}'.format(train_score[0], train_score[1]))
test_score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss: {}, Test accuracy: {}'.format(test_score[0], test_score[1]))

Epoch 1/2
250/250 [==============================] - 414s 2s/step - loss: 5.5432e-09 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/2
250/250 [==============================] - 407s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Train loss: 0.0, Train accuracy: 1.0
Test loss: 0.0, Test accuracy: 1.0


In [17]:
pred = model.predict(X_val)

16/16 [==============================] - 24s 1s/step


In [16]:
from sklearn.metrics import accuracy_score

In [19]:
# np.argmax(pred, axis=1), np.argmax(Y_val, axis=1)
accuracy_score(np.argmax(pred, axis=1), np.argmax(Y_val, axis=1))



1.0

In [22]:
model.save('/content/keras_CNN_model')

In [29]:
!zip -r model.zip /content/keras_CNN_model

  adding: content/keras_CNN_model/ (stored 0%)
  adding: content/keras_CNN_model/keras_metadata.pb (deflated 90%)
  adding: content/keras_CNN_model/saved_model.pb (deflated 88%)
  adding: content/keras_CNN_model/assets/ (stored 0%)
  adding: content/keras_CNN_model/variables/ (stored 0%)
  adding: content/keras_CNN_model/variables/variables.index (deflated 64%)
  adding: content/keras_CNN_model/variables/variables.data-00000-of-00001 (deflated 69%)


In [21]:
import pickle

In [23]:
filename = 'keras_CNN_picklefile'
pickle.dump(model, open(filename, "wb"))

In [ ]:
# import keras
# from keras.layers import Dense, Flatten
# from keras.layers import Conv2D, MaxPooling2D
# from keras.models import Sequential
# from keras.callbacks import History 
# history = History()
 
# img_x = 1249
# img_y = 1249
# img_z = 8
# input_shape = (img_x, img_y, img_z)
 
# num_classes = 6
# batch_size = 16
# num_classes = 7
# epochs = 10
 
# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
 
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)
 
 
# model = Sequential()
# model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
#                  activation='relu',
#                  input_shape=input_shape))
# model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# model.add(Conv2D(64, (5, 5), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Flatten())
# model.add(Dense(1000, activation='relu'))
# model.add(Dense(num_classes, activation='softmax'))
 
# model.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.Adam(),
#               metrics=['accuracy'])
 
 
# model.fit(x_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(x_test, y_test),
#           callbacks=[history])
 
# train_score = model.evaluate(x_train, y_train, verbose=0)
# print('Train loss: {}, Train accuracy: {}'.format(train_score[0], train_score[1]))
# test_score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss: {}, Test accuracy: {}'.format(test_score[0], test_score[1]))